# Preço teto

O código Python carrega dados de dois arquivos CSV, dividendos_ano_resumo.csv e preco_acoes.csv, e calcula o preço teto de ações com base nos dividendos. Ele adiciona o sufixo .SA aos tickers do primeiro arquivo para alinhá-los com o segundo, realiza um merge para combinar os dados, e calcula o preco_teto_5anos dividindo o valor_5anos por 5 e multiplicando por 100/6. Em seguida, computa a diferenca_percentual entre o preco_teto_5anos e o fechamento_atual, expressa em porcentagem, garantindo que valores nulos ou divisões por zero sejam tratados como NaN. O resultado, com as colunas ticker, preco_teto_5anos e diferenca_percentual, é salvo em um novo CSV chamado preco_teto.csv, e as cinco primeiras linhas são exibidas.

In [7]:
import pandas as pd

# Definir a porcentagem de dividendo para cálculo do preço teto
dividendo_porcentagem = 100 / 6

# Carregar os dados dos CSVs
resumo = pd.read_csv("../data/dividendos_ano_resumo.csv")
preco_acoes = pd.read_csv("../data/precos_acoes.csv")

# Adicionar o sufixo .SA aos tickers de dividendos_ano_resumo.csv
resumo['ticker'] = resumo['ticker'] + '.SA'

# Fazer o merge dos dados com base no ticker
dados = resumo.merge(preco_acoes, on='ticker', how='left')

# Calcular o preço teto
# Preço teto 5 anos = (valor_5anos / 5) * dividendo_porcentagem
dados['preco_teto_5anos'] = ((dados['valor_5anos'] / 5) * dividendo_porcentagem).round(2)

# Calcular a diferença percentual: ((preco_teto_5anos - fechamento_atual) / fechamento_atual) * 100
# Evitar divisão por zero e lidar com NaN
dados['diferenca_percentual'] = dados.apply(
    lambda row: round(((row['preco_teto_5anos'] - row['fechamento_atual']) / row['fechamento_atual'] * 100), 2)
    if pd.notnull(row['preco_teto_5anos']) and pd.notnull(row['fechamento_atual']) and row['fechamento_atual'] != 0
    else float('nan'),
    axis=1
)

# Selecionar apenas as colunas necessárias
resultado = dados[['ticker', 'preco_teto_5anos', 'diferenca_percentual']]

# Salvar o resultado em um novo CSV
resultado.to_csv('../data/preco_teto.csv', index=False)
print("Arquivo 'preco_teto.csv' gerado com sucesso!")

# Mostrar as 5 primeiras linhas do resultado
print(resultado.head(5))

Arquivo 'preco_teto.csv' gerado com sucesso!
      ticker  preco_teto_5anos  diferenca_percentual
0   ABCB4.SA             23.01                  6.68
1   AGRO3.SA             42.15                107.23
2   ALOS3.SA             10.48                -53.07
3  ALUP11.SA             17.70                -39.11
4   ALUP3.SA              5.90                -42.83
